In [15]:
import sqlite3

connection = sqlite3.connect('L03V2.db')
cursor = connection.cursor()   

In [16]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('eco',), ('city',), ('Citty',), ('Country',), ('Language',), ('EcoInfo',)]


# M1

### Tablas
City:
**ID**,
CountryCode,
Name,
District,
Population

Country:
**CountryCode**,
Continent,
Region,
CountryName,
CountrySurface,
IndepYear,
CountryPopulation,
CountryLifeExpectancy,
GNP,
GNPOld,
CountryLocalName,
HeadOfState,
HDI,
GDP,
GovernmentForm

Language:
**CountryCode**,
**Language**,
IsLanguageOfficial,
LanguagePercentage

EcoInfo:
**CountryCode**,
Cropland Footprint,
Grazing Footprint,
Forest Footprint,
Carbon Footprint,
Fish Footprint,
Total Ecological Footprint,
Cropland,
Grazing Land,
Forest Land,
Fishing Water,
Urban Land,
Total Biocapacity,
Biocapacity Deficit or Reserve,
Data Quality

In [17]:
# Creamos las ciudades en caso de que no esten. Sino las borramos y las creamos. 

create_city = """CREATE TABLE Citty (
    ID INTEGER PRIMARY KEY,
    Capital TEXT,
    CountryCode TEXT,
    Name TEXT,
    District TEXT,
    Population INTEGER
);"""

create_country = """CREATE TABLE Country (
    CountryCode TEXT PRIMARY KEY,
    Continent TEXT,
    Region TEXT,
    CountryName TEXT,
    CountrySurface REAL,
    IndepYear INTEGER,
    CountryPopulation INTEGER,
    CountryLifeExpectancy REAL,
    GNP REAL,
    GNPOld REAL,
    CountryLocalName TEXT,
    HeadOfState TEXT,
    HDI REAL,
    GDP REAL,
    GovernmentForm TEXT
);"""

create_language = """CREATE TABLE Language (
    CountryCode TEXT,
    Language TEXT,
    IsLanguageOfficial TEXT,
    LanguagePercentage REAL,
    PRIMARY KEY (CountryCode, Language)
);"""

create_eco = """CREATE TABLE EcoInfo (
    CountryCode TEXT PRIMARY KEY,
    CroplandFootprint REAL,
    GrazingFootprint REAL,
    ForestFootprint REAL,
    CarbonFootprint REAL,
    FishFootprint REAL,
    TotalEcologicalFootprint REAL,
    CroplandLand REAL,
    GrazingLand REAL,
    ForestLand REAL,
    FishingWater REAL,
    UrbanLand REAL,
    TotalBiocapacity REAL,
    BiocapacityDeficitorReserve REAL,
    DataQuality REAL
);"""

try:
    cursor.execute("DROP TABLE Citty")
    cursor.execute(create_city)
except:
    cursor.execute(create_city)

try:
    cursor.execute("DROP TABLE Country")
    cursor.execute(create_country)
except:
    cursor.execute(create_country)

try:
    cursor.execute("DROP TABLE Language")
    cursor.execute(create_language)
except:
    cursor.execute(create_language)

try:
    cursor.execute("DROP TABLE EcoInfo")
    cursor.execute(create_eco)
except:
    cursor.execute(create_eco)

In [18]:
# agregamos la información a las tablas.

def to_f(v, f):
    if v in ['', 'NULL']:
        return "NULL"
    elif f == "GDP":
        return float(v[1:].replace(',',''))
    else:
        return f(v)

# para cada pais
cursor.execute("SELECT DISTINCT CountryCode FROM city")
for code in cursor.fetchall():

    # obtenemos la información de cada lengauge del pais
    cursor.execute(f"SELECT DISTINCT CountryCode, Language, IsLanguageOfficial, LanguagePercentage FROM city WHERE CountryCode='{code[0]}'")
    for CountryCode, Language, IsLanguageOfficial, LanguagePercentage in cursor.fetchall():
        cursor.execute(f"INSERT INTO Language VALUES ('{CountryCode}','{Language}', '{IsLanguageOfficial}', {float(LanguagePercentage)})")

    # obtenemos la informacion de cada ciudad del pais
    cursor.execute(f"SELECT DISTINCT ID, Capital, CountryCode, Name, District, Population FROM city WHERE CountryCode='{code[0]}'")
    for ID, Capital, CountryCode, Name, District, Population in cursor.fetchall():
        cursor.execute(f"INSERT INTO Citty VALUES ('{ID}','{str(ID==Capital)}', '{CountryCode}', '{Name}', '{District}', {int(Population)})")

    # obtenemos la informacion del pais
    cursor.execute(f"SELECT DISTINCT CountryCode, Continent, Region, CountryName, CountrySurface, IndepYear, CountryPopulation, CountryLifeExpectancy, GNP, GNPOld, CountryLocalName, HeadOfState, GovernmentForm FROM city WHERE CountryCode='{code[0]}'")
    CountryCode, Continent, Region, CountryName, CountrySurface, IndepYear, CountryPopulation, CountryLifeExpectancy, GNP, GNPOld, CountryLocalName, HeadOfState, GovernmentForm = cursor.fetchone()

    # obtenemos la informacion economica de cada pais
    cursor.execute(f"""SELECT DISTINCT HDI, "GDP per Capita", "Cropland Footprint", "Grazing Footprint", "Forest Footprint", "Carbon Footprint", "Fish Footprint", "Total Ecological Footprint", Cropland, "Grazing Land", "Forest Land", "Fishing Water", "Urban Land", "Total Biocapacity", "Biocapacity Deficit or Reserve", "Earths Required", "Countries Required", "Data Quality" FROM eco WHERE Country="{CountryName}" """)
    data = cursor.fetchone()
    if data is not None:
        HDI, GDP, CroplandFootprint, GrazingFootprint, ForestFootprint, CarbonFootprint, FishFootprint, TotalEcologicalFootprint, CroplandLand, GrazingLand, ForestLand, FishingWater, UrbanLand, TotalBiocapacity, BiocapacityDeficitorReserve, EarthsRequired, CountriesRequired, DataQuality = data
        cursor.execute(f"""INSERT INTO EcoInfo VALUES ('{CountryCode}', {to_f(CroplandFootprint, float)}, {to_f(GrazingFootprint, float)}, {to_f(ForestFootprint, float)}, {to_f(CarbonFootprint, float)}, {to_f(FishFootprint, float)}, {to_f(TotalEcologicalFootprint, float)}, {to_f(CroplandLand, float)}, {to_f(GrazingLand, float)}, {to_f(ForestLand, float)}, {to_f(FishingWater, float)}, {to_f(UrbanLand, float)}, {to_f(TotalBiocapacity, float)}, {to_f(BiocapacityDeficitorReserve, float)}, '{DataQuality}')""")
    else:
        HDI = ''
        GDP = ''

    cursor.execute(f"""INSERT INTO Country VALUES ('{CountryCode}', '{Continent}', '{Region}', '{CountryName}', {to_f(CountrySurface, float)}, {to_f(IndepYear, int)}, {to_f(CountryPopulation, int)}, {to_f(CountryLifeExpectancy, float)}, {to_f(GNP, float)}, {to_f(GNPOld, float)}, '{CountryLocalName}', '{HeadOfState}', {to_f(HDI, float)}, {to_f(GDP, "GDP")}, '{GovernmentForm}')""")

# M2

In [19]:
def query(q):
    cursor.execute(q)
    return cursor.fetchall()

In [20]:
# a)
# obtenemos todos los lengauges que son oficial en alguna parte del mundo
langOf = "SELECT DISTINCT Language AS Lang FROM Language WHERE IsLanguageOfficial = 'T'"

# obtenemos la cantidad del paises que hablan cada idioma
q = """SELECT Language, COUNT(CountryCode) 
FROM Language 
GROUP BY Language"""

print(query(q))

print()

# obtenemos la cantidad del paises que hablan cada idioma, si este idioma es oficial en algun pais
q = f"""
SELECT L.language, COUNT(L.countrycode)
FROM Language AS L
WHERE L.language IN ({langOf})
GROUP BY Language"""

print(query(q))

[('Abhyasi', 1), ('Acholi', 1), ('Adja', 1), ('Afar', 2), ('Afrikaans', 2), ('Aimará', 3), ('Ainu', 1), ('Aizo', 1), ('Akan', 2), ('Albaniana', 4), ('Ambo', 1), ('Amhara', 1), ('Ami', 1), ('Ane', 1), ('Arabic', 33), ('Arabic-French', 1), ('Arabic-French-English', 1), ('Araucan', 1), ('Arawakan', 2), ('Armenian', 5), ('Asami', 2), ('Assyrian', 1), ('Atayal', 1), ('Avarian', 1), ('Azerbaijani', 5), ('Bajad', 1), ('Bajan', 1), ('Bakhtyari', 1), ('Balante', 1), ('Bali', 1), ('Balochi', 4), ('Bambara', 1), ('Bamileke-bamum', 1), ('Banda', 1), ('Banja', 1), ('Bari', 1), ('Bariba', 1), ('Bashkir', 1), ('Basque', 1), ('Bassa', 1), ('Batakki', 1), ('Beja', 1), ('Belorussian', 7), ('Bemba', 1), ('Bengali', 2), ('Berberi', 3), ('Bhojpuri', 2), ('Bicol', 1), ('Bilin', 1), ('Bislama', 1), ('Brahui', 1), ('Bubi', 1), ('Bugi', 1), ('Bulgariana', 3), ('Bullom-sherbro', 1), ('Bura', 1), ('Burmese', 1), ('Buryat', 1), ('Busansi', 1), ('Cakchiquel', 1), ('Canton Chinese', 3), ('Caprivi', 1), ('Caribbean'

In [21]:
# b)

#para cada pais, determinamos cuantos lenguages se hablan por mas de 40% de la poblacion
per = """
SELECT CountryCode AS Cc, COUNT(Language) as Tot 
FROM Language 
WHERE LanguagePercentage > 40 
GROUP BY CountryCode"""

#para cada pais, determinamos cuantos lenguages se hablan
cou = """
SELECT CountryCode AS Cc, COUNT(Language) as Tot 
FROM Language 
GROUP BY CountryCode"""

#revisamos que para cada pais, la cantidad de idiomas sea igual a la cantidad de idiomas sobre 40%
main = f"""
SELECT COUNT(Per.Cc)
FROM ({per}) AS Per, ({cou}) AS Cou 
WHERE Per.Cc=Cou.Cc AND Per.Tot=Cou.Tot"""

print(query(main))

per = """
SELECT CountryCode AS Cc, COUNT(Language) as Tot 
FROM Language 
WHERE LanguagePercentage > 40 AND IsLanguageOfficial = 'T' 
GROUP BY CountryCode"""

cou = """
SELECT CountryCode AS Cc, COUNT(Language) as Tot 
FROM Language 
WHERE IsLanguageOfficial = 'T' 
GROUP BY CountryCode"""

main = f"""
SELECT COUNT(Per.Cc) 
FROM ({per}) AS Per, ({cou}) as Cou 
WHERE Per.Cc=Cou.Cc AND Per.Tot=Cou.Tot"""

print(query(main))

[(11,)]
[(108,)]


In [22]:
# c)

# buscamos todas las ciudades con mas de 1 millon de personas por pais
over = """
SELECT CountryCode AS Cc, COUNT(*) AS Tot
FROM Citty as C
WHERE Population > 1000000
GROUP BY CountryCode"""

# contamos todas las ciudades por pais
total = """
SELECT CountryCode AS Cc, COUNT(*) AS Tot
FROM Citty as C
GROUP BY CountryCode"""

# para cada pais, vemos si este tiene mas ciudades que 10*(1 millon)
main = f"""
SELECT COUNT(O.Cc)
FROM ({over}) AS O, ({total}) AS T
WHERE O.Cc = T.Cc AND T.Tot/10 < O.Tot"""

print(query(main))

[(27,)]


In [23]:
# d)

# buscamos todos los paises que tienen mas de 5 lenguages
pais = """
SELECT CountryCode AS Cc, COUNT(Language) as Tot 
FROM Language 
GROUP BY CountryCode 
HAVING Tot > 5"""

# buscamos los paises que su suelo urbano es mayor al promedio
tierra = """
SELECT CountryCode as Cc, (CroplandLand+GrazingLand+ForestLand+FishingWater+UrbanLand)*0.2 as Tot, UrbanLand 
FROM EcoInfo 
WHERE UrbanLand IS NOT NULL AND UrbanLand > Tot"""

# sacamos todos los paises que pertenecena ambos queries anteriores
main = f"""
SELECT COUNT(P.Cc)
FROM ({pais}) as P
INNER JOIN ({tierra}) as T
ON P.Cc = T.Cc"""

print(query(main))

[(1,)]


# M3

In [24]:
# a) vamos a tomar riqueza como GDP*poblacion

# obtenemos la riqueza de cada region 
reg = """
SELECT Region AS Reg, SUM(GDP*CountryPopulation) AS Tot 
FROM Country
GROUP BY Region
"""

# sacamos la riqueza de la region mas pobre
auxReg = f"""
SELECT MIN(Tot) as Min 
FROM ({reg})
"""

# obtenemos el nombre de la region mas pobre
region = f"""
SELECT Reg
FROM ({reg})
WHERE Tot = ({auxReg})
"""

# obtenemos la riqueza de cada pais en la region mas pobre
cntry = f"""
SELECT CountryName, GDP*CountryPopulation AS Tot 
FROM Country
WHERE Region = ({region}) AND GDP IS NOT NULL AND CountryPopulation IS NOT NULL
"""

# obtenemos la riqueza maxima entre los paises
auxCntry = f"""
SELECT MAX(Tot) as Min 
FROM ({cntry})
"""

# obtenemos el nombre del pais
country = f"""
SELECT CountryName
FROM ({cntry})
WHERE Tot = ({auxCntry})
"""

print(query(country))

[('Samoa',)]


In [25]:
# b)

# obtenemos la riqueza de cada region 
reg = """
SELECT Region AS Reg, SUM(GDP*CountryPopulation) AS Tot 
FROM Country
GROUP BY Region
"""

# sacamos la riqueza de la region mas pobre
auxReg = f"""
SELECT MIN(Tot) as Min 
FROM ({reg})
"""

# obtenemos los paises de la region
region = f"""
SELECT Cntry.CountryCode as Cc
FROM ({reg})as Reg, Country as Cntry
WHERE Reg.Tot = ({auxReg}) AND Cntry.Region = Reg.Reg
"""

# obtenemos las cantidades de suelos
tierra = f"""
SELECT (E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand) AS Tot, E.ForestLand AS Forest
FROM EcoInfo AS E
WHERE E.CountryCode IN ({region})"""

# sacamos el promedio
main = f"""
SELECT SUM(Forest) / SUM(Tot)
FROM ({tierra})
"""

print(query(main))

[(0.4205020920502092,)]


In [26]:
# c)

#obtenemos para cada lenguaje, la cantidad de de personas qeu lo habla
total = f"""
SELECT L.Language AS Lang, SUM(L.LanguagePercentage * C.Population) AS Sum
FROM Language AS L, Citty AS C
GROUP BY Lang
ORDER BY Sum DESC
LIMIT 5
"""

#obtenemos los paises y sus lenguages donde mas de 50% de la pobalcion habla el lenguage
pais = f"""
SELECT CountryCode AS Cc, Language AS Lang
FROM Language
WHERE LanguagePercentage > 50 AND IsLanguageOfficial = 'T'
"""

#obtenemos los paises y lenguages que pertenecen a las dos tablas anteriores
join = f"""
SELECT P.Cc, T.Lang
FROM ({total}) AS T
INNER JOIN ({pais}) as P
WHERE T.Lang = P.Lang
"""

#obtenemos la huella ecologica de cada pais
ecologica = f"""
SELECT CountryCode AS Cc, TotalEcologicalFootprint AS Tef
FROM EcoInfo
"""

#obtenememos los paises que encontramos antes y obtenemos sus huellas ecologicas.
join2 = f"""
SELECT E.Cc AS Cc, MAX(E.Tef) AS Tef, J.Lang AS Lang
FROM ({join}) AS J
INNER JOIN ({ecologica}) as E
WHERE J.Cc = E.Cc
GROUP BY Lang
"""

#obtenemos los nombres y poblaciones de los paises
main = f"""
SELECT C.CountryName, C.CountryPopulation
FROM ({join2}) AS J, Country AS c
WHERE C.CountryCode = J.Cc
ORDER BY C.CountryPopulation DESC
"""

print(query(main))

[('Australia', 18886000), ('Chile', 15211000), ('Kuwait', 1972000)]


In [27]:
# d)


lenguje = f"""
SELECT Ci.Name, L.Language, L.IsLanguageOfficial, L.LanguagePercentage * Co.CountryPopulation AS Tot
FROM Citty AS Ci, Language AS L, Country AS Co
WHERE Ci.CountryCode = Co.CountryCode AND Co.CountryCode = L.CountryCode
GROUP BY Ci.Name
HAVING Tot > 0
ORDER BY Tot ASC
LIMIT 1
"""

print(query(lenguje))

[('Bairiki', 'Tuvalu', 'F', 41500.0)]


# M4

In [28]:
# a)

#obtenemos la poblacion de cada distrito de cada pais
dist = f"""
SELECT Ci.CountryCode AS Cc, Ci.District AS District, SUM(Ci.Population) AS Tot
FROM Citty AS Ci
WHERE Ci.District <> ''
GROUP BY Ci.District
HAVING Tot>0
"""

n = 10

#obtenemos el porcentaje poblacional de los 10 distritos con mas
distritos1 = f"""
SELECT D.District, (D.Tot*100)/C.CountryPopulation AS Tot
FROM ({dist}) AS D, Country AS C
WHERE D.Cc = C.CountryCode AND D.District <> '–'
GROUP BY D.District
ORDER BY Tot DESC
LIMIT {n}
"""


#obtenemos el porcentaje poblacional de los 10 distritos con menos
distritos2 = f"""
SELECT D.District, (D.Tot*100)/C.CountryPopulation AS Tot
FROM ({dist}) AS D, Country AS C
WHERE D.Cc = C.CountryCode AND D.District <> '–'
GROUP BY D.District
ORDER BY Tot ASC
LIMIT {n}
"""

main = f"""
SELECT * FROM ({distritos1})
UNION ALL
SELECT * FROM({distritos2})
"""

print(query(main))

[('Macau', 92), ('Home Island', 83), ('Saint-Pierre', 82), ('East Falkland', 81), ('Santiago', 68), ('Distrito Federal', 64), ('Koror', 63), ('Djibouti', 60), ('Doha', 59), ('Rarotonga', 59), ('ARMM', 0), ('Abruzzit', 0), ('Aceh', 0), ('Acre', 0), ('Adiyaman', 0), ('Adygea', 0), ('Afyon', 0), ('Aguascalientes', 0), ('Akita', 0), ('Aksaray', 0)]


In [29]:
# b)

# vamos a tomar GHA*Poblacion como nivel de contaminacion

#obtenemos el nivel de contaminacion por region de las 5 mas contaminantes
regs = f"""
SELECT C.Region AS Reg, SUM(E.TotalEcologicalFootprint * C.CountryPopulation) AS Tot
FROM Country AS C, EcoInfo AS E
WHERE C.CountryCode = E.CountryCode
GROUP BY C.Region
ORDER BY Tot DESC
"""

#obtenemos las regiones
regiones = f"""
SELECT Reg
FROM ({regs})
"""

#para las regiones que encontramos obtenemos los nombres de los paises y su contaminacion
rr = f"""
SELECT C.Region AS Region, C.CountryName AS Name, E.TotalEcologicalFootprint * C.CountryPopulation AS Tot
FROM Country AS C, EcoInfo AS E
WHERE C.CountryCode = E.CountryCode AND C.Region IN ({regiones})
ORDER BY Tot DESC
"""

#obtenemos los tres paises mas contaminantes por region
main = f"""
SELECT Region, Name, Tot
FROM ({rr}) AS Data
WHERE Name IN (
      SELECT Name
      FROM ({rr})
      WHERE Region = Data.Region
      ORDER BY Tot DESC
      LIMIT 3
)
ORDER BY Region, Tot ASC
"""

print(query(main))

[('Australia and New Zealand', 'New Zealand', 21627200.0), ('Australia and New Zealand', 'Australia', 175828660.0), ('Baltic Countries', 'Estonia', 9872912.0), ('Baltic Countries', 'Latvia', 15248218.0), ('Baltic Countries', 'Lithuania', 21562255.0), ('British Islands', 'Ireland', 21027307.0), ('British Islands', 'United Kingdom', 294539596.0), ('Caribbean', 'Trinidad and Tobago', 10256400.0), ('Caribbean', 'Dominican Republic', 12997350.0), ('Caribbean', 'Cuba', 21841950.0), ('Central Africa', 'Chad', 11170460.0), ('Central Africa', 'Angola', 11976540.0), ('Central Africa', 'Cameroon', 17649450.0), ('Central America', 'El Salvador', 12991319.999999998), ('Central America', 'Guatemala', 21517650.0), ('Central America', 'Mexico', 285766090.0), ('Eastern Africa', 'Uganda', 27004720.0), ('Eastern Africa', 'Kenya', 30982400.0), ('Eastern Africa', 'Ethiopia', 63816300.0), ('Eastern Asia', 'Mongolia', 16184960.0), ('Eastern Asia', 'Japan', 636104280.0), ('Eastern Asia', 'China', 4318146040.0

In [30]:
# c)

#obtenemos las 5 regiones mas ricas 
reg = f"""
SELECT Reg 
FROM (
    SELECT Region AS Reg, SUM(GDP*CountryPopulation) AS Tot 
    FROM Country
    GROUP BY Region
    ORDER BY Tot DESC
    LIMIT 5
)
"""

#para cada region obtenemos los porcentajes de sus usos de suelo
main = f"""
SELECT C.Region AS Reg,
    SUM(C.GDP*C.CountryPopulation) AS Tot, 
    SUM(E.CroplandLand)/SUM(E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand),
    SUM(E.FishingWater)/SUM(E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand),
    SUM(E.ForestLand)/SUM(E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand),
    SUM(E.GrazingLand)/SUM(E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand),
    SUM(E.UrbanLand)/SUM(E.CroplandLand+E.GrazingLand+E.ForestLand+E.FishingWater+E.UrbanLand)
FROM Country AS C, EcoInfo AS E
WHERE C.Region IN ({reg}) AND C.CountryCode = E.CountryCode
GROUP BY C.Region
ORDER BY Tot ASC
"""

print(query(main))

[('Eastern Europe', 3473723165942.0, 0.3787824848700605, 0.05980776076895692, 0.4834460662157351, 0.045567817728729094, 0.032395870416518334), ('South America', 3587054494190.0, 0.033089005235602105, 0.10010471204188484, 0.815198204936425, 0.04837696335078534, 0.0032311144353029176), ('Southern Europe', 4133116147194.0, 0.3487332339791357, 0.05663189269746647, 0.4910581222056632, 0.0745156482861401, 0.029061102831594632), ('Western Europe', 8978556777650.0, 0.39956490210297313, 0.04786076867295141, 0.4111675126903553, 0.04713560551124003, 0.09427121102248007), ('Eastern Asia', 12968758345260.0, 0.04794916233391103, 0.016175621028307337, 0.439052570768342, 0.4812247255921432, 0.015597920277296359)]


In [31]:
# d)

# riqueza = gdp*lifeexpectancy

#obtenemos la riqueza de cada pais
reg = """
SELECT Region AS Reg, CountryName AS Name, GDP*CountryLifeExpectancy AS Tot 
FROM Country
WHERE GDP IS NOT NULL AND CountryLifeExpectancy IS NOT NULL
"""

#obtenemos los 5 paises mas rico para cada region
main = f"""
SELECT Reg, Name, Tot
FROM ({reg}) AS Data
WHERE Name IN (
      SELECT Name
      FROM ({reg})
      WHERE Reg = Data.Reg
      ORDER BY Tot DESC
      LIMIT 5
)
ORDER BY Reg, Tot ASC
"""

print(query(main))

[('Australia and New Zealand', 'New Zealand', 2916589.74), ('Australia and New Zealand', 'Australia', 5315015.159999999), ('Baltic Countries', 'Latvia', 939268.8), ('Baltic Countries', 'Lithuania', 993222.6699999999), ('Baltic Countries', 'Estonia', 1202655.8), ('British Islands', 'United Kingdom', 3184184.85), ('British Islands', 'Ireland', 4059156.4799999995), ('Caribbean', 'Saint Kitts and Nevis', 915741.75), ('Caribbean', 'Antigua and Barbuda', 930959.55), ('Caribbean', 'Barbados', 1147764.4), ('Caribbean', 'Trinidad and Tobago', 1245134.4), ('Caribbean', 'Bahamas', 1610223.0299999998), ('Central Africa', 'Sao Tome and Principe', 90409.809), ('Central Africa', 'Congo', 92130.906), ('Central Africa', 'Angola', 178704.35299999997), ('Central Africa', 'Gabon', 600648.9), ('Central Africa', 'Equatorial Guinea', 1289830.4000000001), ('Central America', 'Guatemala', 214077.56000000003), ('Central America', 'El Salvador', 257634.201), ('Central America', 'Panama', 635013.135), ('Central A

# M5 (BONUS)

In [32]:
# a)

# para cada una de las nuevas columnas, las agregamos a la tabla citty
alter = "ALTER TABLE Citty ADD COLUMN {} REAL DEFAULT NULL"
for i in ['CroplandFootprint','GrazingFootprint','ForestFootprint','CarbonFootprint','FishFootprint','TotalEcologicalFootprint']:
    cursor.execute(alter.format(i))
    
cursor.execute("SELECT * FROM sqlite_master WHERE type='table' AND name='Citty';")
print(cursor.fetchall())

[('table', 'Citty', 'Citty', 61, 'CREATE TABLE Citty (\n    ID INTEGER PRIMARY KEY,\n    Capital TEXT,\n    CountryCode TEXT,\n    Name TEXT,\n    District TEXT,\n    Population INTEGER\n, CroplandFootprint REAL DEFAULT NULL, GrazingFootprint REAL DEFAULT NULL, ForestFootprint REAL DEFAULT NULL, CarbonFootprint REAL DEFAULT NULL, FishFootprint REAL DEFAULT NULL, TotalEcologicalFootprint REAL DEFAULT NULL)')]


In [33]:
# obtenemos todos los valores y los agregamos a la tabla
qq = f"""
SELECT C.ID,C.Capital,C.CountryCode,C.Name,C.District,C.Population,
    E.CroplandFootprint * C.Population, E.GrazingFootprint * C.Population,
    E.ForestFootprint * C.Population, E.CarbonFootprint * C.Population,
    E.FishFootprint * C.Population, E.TotalEcologicalFootprint * C.Population
FROM EcoInfo AS E, Citty AS C
WHERE E.CountryCode = C.CountryCode
"""

main = f"""
REPLACE INTO Citty (ID,Capital,CountryCode,Name,District,Population,CroplandFootprint,GrazingFootprint,ForestFootprint,CarbonFootprint,FishFootprint,TotalEcologicalFootprint)
{qq}
"""

cursor.execute(main)

In [34]:
#revisamos que todo esta ok

main = f"""
SELECT * FROM Citty LIMIT 2
"""

print(query(main))

[(1, 'True', 'AFG', 'Kabul', 'Kabol', 1780000, 534000.0, 356000.0, 142400.0, 320400.0, 0.0, 1406200.0), (2, 'False', 'AFG', 'Qandahar', 'Qandahar', 237500, 71250.0, 47500.0, 19000.0, 42750.0, 0.0, 187625.0)]


In [35]:
# b)
# buscamos los 20 paises mas contaminantes
countries = f"""
SELECT C.CountryCode AS Cc
FROM Country AS C, EcoInfo AS E
WHERE C.CountryCode = E.CountryCode
ORDER BY E.TotalEcologicalFootprint DESC
LIMIT 20
"""

# obtenemos las ciudades y lo que contaminan
ciudades = f"""
SELECT Ci.Name AS Name, Co.Continent AS Continent, Ci.TotalEcologicalFootprint AS FootPrint
FROM Citty AS Ci, Country AS Co
WHERE Ci.CountryCode = Co.CountryCode AND Ci.CountryCode IN ({countries})
"""

# para cada continente escogemos las 3 mas contaminantes
main = f"""
SELECT Name, Continent, FootPrint
FROM ({ciudades}) AS Data
WHERE Name IN (
      SELECT Name
      FROM ({ciudades})
      WHERE Continent = Data.Continent
      ORDER BY FootPrint DESC
      LIMIT 3
)
ORDER BY Continent, FootPrint
"""

print(query(main))

[('Ulan Bator', 'Asia', 4704096.0), ('Dubai', 'Asia', 5306605.33), ('Singapore', 'Asia', 32021332.009999998), ('Riga', 'Europe', 4807623.12), ('Stockholm', 'Europe', 5440023.0), ('Wien', 'Europe', 9745352.639999999), ('Toronto', 'North America', 5623206.75), ('Calgary', 'North America', 6275229.94), ('Montréal', 'North America', 8303791.92), ('Brisbane', 'Oceania', 12020299.270000001), ('Melbourne', 'Oceania', 26676212.990000002), ('Sydney', 'Oceania', 30501487.17)]
